In [10]:
%matplotlib widget

In [11]:
# needed libraries
import warnings
warnings.simplefilter("ignore")

import math
import numpy
import scipy
import cmath

from scipy import integrate

import matplotlib.pyplot as plt

import ipywidgets as widgets

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (8, 8),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)



In [12]:
# A) Initialisation des variables  

# 1. Constantes physiques (NIST)
h    = 6.626068e-34  # [J.S]      Planck constant
heV  = 4.135667e-15  # [eV.s]     Planck constant
hb   = h/(2*math.pi)      # [J.s]      h barre
hbeV = heV/(2*math.pi)    # [eV.s]     h barre
me   = 9.109381e-31  # [kg]       electron mass
q    = 1.6021764e-19 # [C]        electron charge
c    = 299792458     # [m.s^(-1)] speed of light in vacuum

# 2. Propriété de la particule étudiée
m = 0.1*me  # [kg] masse (me = masse de l'électron)
a  = 5e-9   #    extension spatiale du paquet d'onde                        
x0 = -60e-9 # position centrale du pic de probabilité
                              
# 3. Propriétés de l'espace étudié (puits, maille,...)
xMIN = -100e-9 # [m] extension maximale vers la gauche
xMAX = -xMIN   # [m] extension maximale vers la droite
xNum = 601     #  [-] nombre de valeurs discrétisées de x (résolution) doit être impair
max_iter = xNum/2    # nombre d'itérations par pas de dt

# 5 Définitions de constantes internes
dx    = (xMAX-xMIN)/(xNum-1) # [m]
xArr  = numpy.linspace(xMIN, xMAX, xNum-1)
xOnes = [1.0] * (xNum-1)


In [13]:
# B) Définition du potentiel ####################################################################################

def Potentiel(idxPot,PotMAX,Width,Width2):
    """"Args:  
            - idxPot: entier pour la sélection du potentiel 
            - PotMax: hauteur, 
            - Width: largeur, 
            - Witdth2: écart
          Return:
            - le potentiel"""

    if idxPot==2: # marche 
        PotMAX = PotMAX*q
        Pot1 = [0] * int((xNum-1)/2)
        Pot2 = [PotMAX] * int((xNum-1)/2)
        Pot = Pot1 + Pot2
    elif idxPot==3: # barrière centrée en 0
        PotMAX = PotMAX*q
        Width = Width*1e-9
        Pot = []
        for ii, xx in enumerate(xArr):
            if (xx <= - Width/2) or (xx >= Width/2):
                Pot.append(0.0)
            else:
                Pot.append(PotMAX)
    elif idxPot==4: # double barriére
        PotMAX = PotMAX*q
        Width = Width*1e-9
        Width2 = Width2*1e-9
        Pot = []
        for ii, xx in enumerate(xArr):
            if (xx <= -(Width+Width2)/2) or (xx > -Width2/2) and (xx < Width2/2) or (xx >= (Width+Width2)/2):
                Pot.append(0.0)
            else:
                Pot.append(PotMAX)
    else: 
        PotMAX = q
        Pot = [0.0] * int(xNum-1)
    return Pot


In [14]:
def Schroedinger_Dep(energie_p,PotMAX,Pot,on_off):
    """"Args:  
            - energie_p: énergie de la particule 
            - PotMAX: hauteur, 
            - Pot: potential, 
            - on_off: calcule les matrices M et L
          Return:
            - n_Psi2_norm_list: la fonction d'onde normée"""

    #énergie de la particule
    E = energie_p*PotMAX*q
    # Fonction d'onde
    k0 = math.sqrt(2*m*E)/hb  #  nombre d'onde moyen
    # fonction d'onde
    Psi = [ (cmath.exp(1j*xx*k0) * cmath.exp(-((xx-x0)/a)**2)) for xx in xArr]
    Psi = numpy.array(Psi)
    # norme de la fonction d'onde = densité
    Psi2 = numpy.conj(Psi)*Psi
    # intégration de la densité 
    
    Psi_int = integrate.simps(Psi2, x=xArr)
    # normalistation de la densité
    Psi2_norm = Psi2 / Psi_int.real
    PSIMAX = max(Psi2_norm.real)
   
    # 4. Déplacement du paquet d'onde
    #  Quelques variables
    vg   = hb*k0/m       # [m.s^(-1)] vitesse de groupe
    tMAX = 3*abs(x0)/vg  # [s]        temps total = 3 fois le tps nécessaire pour parcourir x0 à la vitesse vg
    tNum = 1*abs(x0)/dx  # nombre de valeurs discrétisées de t 
    dt   = tMAX/(tNum-1) # [s]
  
    
    n_Psi2_norm_list = [Psi2_norm/PSIMAX]*xNum
    
    if on_off==True:
        #  Definition des matrices M et L
        n_Psi2_norm_list = []
        CI = 4*m*(dx**2)/(hb*dt)
        CR = 2*m*(dx**2)/(hb**2)
        diagM     = 1j*CI - ( CR * numpy.array(Pot) + 2)
        diagL     = 1j*CI + ( CR * numpy.array(Pot) + 2)
        diag1 = [1.0] * int(xNum-2)
        diag2 = [-1.0] * int(xNum-2)
   
        matM = numpy.diag(diagM,k=0) + numpy.diag(diag1,k=1) + numpy.diag(diag1,k=-1)
        matL = numpy.diag(diagL,k=0) + numpy.diag(diag2,k=1) + numpy.diag(diag2,k=-1)
       
        matM_inv = numpy.linalg.inv(matM)

        n_Psi = Psi
        for ii in numpy.arange(1,max_iter):
            n_Psi     = numpy.dot(numpy.dot(matM_inv,matL),n_Psi) # nouvelle fonction d'onde
            n_Psi2    = numpy.conj(n_Psi)*n_Psi #  Distribution de probabilité
            n_Psi_int   = integrate.simps(n_Psi2, x=xArr) # intégration de Simpsons
            n_Psi2_norm = n_Psi2 / n_Psi_int.real # normalisation
            n_Psi2_norm_list.append(n_Psi2_norm/PSIMAX)
 
    return n_Psi2_norm_list
 

In [15]:
def interactive_pot(potentiel,hauteur,largeur,largeur2,energie_p,on_off):
    
    """Graph with interactive widgets: interactive part"""
   
    pot = Potentiel(idxPot=potentiel,PotMAX=hauteur,Width=largeur,Width2=largeur2)
    n_Psi2_norm_list = Schroedinger_Dep(energie_p=energie_p,PotMAX=hauteur,Pot=pot,on_off=on_off)
    
    play = widgets.Play(interval=max_iter,value=0,min=0,max=max_iter,step=1,description="Press play",disabled=False)
    time = widgets.IntSlider(value=0,min=0, max=max_iter, step=1, continuous_update=True)
    widgets.jslink((play, 'value'), (time, 'value'))
    box9 = widgets.HBox([widgets.Label('Temps'),time,play])

    fig = plt.figure(1,figsize=(8,6))
    fig.canvas.layout.min_width = '200px'
    fig.canvas.layout.min_height = '200px'
    
    fig.clf()

    plt.plot(xArr,numpy.array(pot)/q,label='Potentiel', color='b')
    plt.xlabel('Distance (m)')
    plt.ylabel('Potentiel (bleu) - Psi (vert)')
                         
    graph = plt.plot(xArr, n_Psi2_norm_list[time.value],color='g')
    
    def update_lines(change):
        graph[0].set_data(xArr, n_Psi2_norm_list[change.new])
        fig.canvas.draw()
        #fig.canvas.flush_events()
        #fig.canvas.draw_idle()

    time.observe(update_lines, names='value')
    ui2 = widgets.VBox([box9, fig.canvas])
    display(ui2)

In [16]:


"""Graph with interactive widgets: passive part"""


potentiel = widgets.Select(
    options=[('1. Plat (= puits infini) -- pas de variable',1),('2. Marche (hauteur (eV)) ',2),('3. Barrière centrée sur x = 0 (hauteur (eV) et largeur (nm))',3),('4. Double barrière (hauteur (eV) et largeur (nm))',4)],
    disabled=False,layout=widgets.Layout(width='350px', height='150px'))

hauteur = widgets.FloatSlider(value=1,min=0.0001, max=2, step=0.001, continuous_update=False,orientation='vertical')
largeur = widgets.FloatSlider(value=10,min=0, max=100, step=1, continuous_update=False,orientation='vertical')
largeur2 = widgets.FloatSlider(value=10,min=0, max=200, step=1, continuous_update=False,orientation='vertical')
energie_p = widgets.FloatSlider(value=1,min=0.1, max=5, step=0.1, continuous_update=False,orientation='vertical')

nEn = widgets.IntSlider(min=1, max=10, step=1, continuous_update=False,orientation='vertical')
nPsi = widgets.IntSlider(min=1, max=10, step=1, continuous_update=False,orientation='vertical')

on_off = widgets.ToggleButtons(options=[('Hold',False),('Compute',True)],disabled=False,button_style='Success',
       layout=widgets.Layout(width='150px', height='150px'), icon='check',orientation='vertical'  )                      


box1 = widgets.VBox([widgets.Label('Choix du potentiel'),potentiel])
box2 = widgets.VBox([widgets.Label('Hauteur (eV)'),hauteur])
box3 = widgets.VBox([widgets.Label('Largeur (nm)'),largeur])
box4 = widgets.VBox([widgets.Label('Ecart (nm)'),largeur2])
box5 = widgets.VBox([widgets.Label('E P (%hauteur)'),energie_p])
box8 = widgets.VBox([widgets.Label('Simulation'),on_off])

ui1 = widgets.HBox([box1, box2, box3, box4, box5, box8])

out = widgets.interactive_output(interactive_pot,{'potentiel':potentiel,'hauteur':hauteur,'largeur':largeur,'largeur2':largeur2,'energie_p':energie_p,'on_off':on_off})


display(ui1,out)


Output()